# Load API client

In [ ]:
from mpcontribs.client import load_client

In [ ]:
headers = { # just for local testing, kong+apikey will take care of headers
    'X-Consumer-Groups': 'my_project',
    'X-Consumer-Username': 'patrick@the-huck.com',
    #'X-Consumer-Username': 'phuck@lbl.gov'
}
client = load_client(headers=headers)
project = 'my_project'

In [ ]:
dir(client)

# Retrieve and update project

In [ ]:
dir(client.projects)

In [ ]:
client.projects.get_entry(pk=project, _fields=['_all']).response().result

In [ ]:
client.projects.update_entry(pk=project, project={
     #'title': 'Best Title', 'other': {'hello': 'mars', 'a': {'b': '6 %', 'c': '3.44 meV'}},
     'urls': {'PRC': 'https://google.wf'}
}).response().result

In [ ]:
# TODO HierarchicalData json-human

# Create, retrieve, and update contributions

In [ ]:
dir(client.contributions)

In [ ]:
identifier = 'mp-1002'
client.contributions.create_entry(contribution={
    'project': project, 'identifier': identifier,
    'data': {'formula': 'CO2', 'E': '3.33 eV', 'E|V': {'a': 1, 'b': '3 cm'}}
}).response().result

In [ ]:
client.contributions.get_entries(project=project, _fields=['_all']).response().result

In [ ]:
client.contributions.update_entry(
    pk='5dfff6783cb42e1673596b1f', contribution={
        'data': {'formula': 'H2', 'E': '14 eV'},
        'identifier': 'mp-1004'
    }
).response().result

In [ ]:
# TODO HierarchicalData json-human

# Tables

In [ ]:
import pandas_datareader.data as web
from mpcontribs.io.core.components.tdata import Table

In [ ]:
kwargs = dict(contribution='5dfff6783cb42e1673596b1f', name='Dow-Jones Index')
table = Table(web.DataReader('^DJI', 'stooq'), **kwargs)

In [ ]:
tt = Table(table.head(), **kwargs)  # submitting head for testing

In [ ]:
client.tables.create_entry(table=tt.to_dict()).response().result

In [ ]:
client.tables.get_entry(
    pk='5dffffa9c6784937c400dd68', _fields=['_all']#, data_per_page=3, data_page=5
).response().result

In [ ]:
client.tables.update_entry(pk='5dff21eb528af4071b42a52f', table=table.to_dict()).response().result

In [ ]:
client.tables.update_entry(pk='5dffffa9c6784937c400dd68', table={'name': 'DJI Head'}).response().result

In [ ]:
client.tables.delete_entry(pk='5dff21eb528af4071b42a52f').response().result

In [ ]:
# TODO Table Backgrid, Plot Plotly

# Structures

In [ ]:
from pymatgen import Structure, MPRester

In [ ]:
with MPRester() as m:
    structure = m.get_structure_by_material_id("mp-1234")

In [ ]:
sdct = structure.as_dict()
kwargs = dict(contribution='5dfff6783cb42e1673596b1f', name='LuAl2')
sdct.update(kwargs)

In [ ]:
client.structures.create_entry(structure=sdct).response().result

In [ ]:
client.structures.get_entry(pk='5e0003dbbdb336d3aea7f497', _fields=['_all']).response().result

In [ ]:
client.structures.update_entry(pk='5e0003dbbdb336d3aea7f497', structure={'name': 'Lu Al2'}).response().result

In [ ]:
# TODO structure viewer

# Add overview graph

In [ ]:
import plotly.express as px

In [ ]:
contribs = client.contributions.get_entries(
    project='MnO2_phase_selection',
    _fields=['id', 'identifier', 'data.ΔH.value']
).response().result['data']

# Publish and/or delete project

In [ ]:
client.contributions.delete_entry(pk='5dfbed6dccd6c89d21994026').response().result

In [ ]:
client.projects.delete_entry(pk='my_project').response().result

In [ ]:
# use update_entries to also publish according contributions, tables, and structures